In [1]:
import os
import numpy as np
from transformers import AutoFeatureExtractor, AutoModelForImageClassification
import torch
from shark_turbine.aot import *
from iree.compiler.ir import Context
from iree.compiler.api import Session
from datasets import load_dataset
import PIL

C:\Users\zjgar\code\SHARK-Turbine\SHARK-Turbine.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load feature extractor and pretrained model from huggingface
extractor = AutoFeatureExtractor.from_pretrained("microsoft/resnet-18")
model = AutoModelForImageClassification.from_pretrained("microsoft/resnet-18")

C:\Users\zjgar\code\SHARK-Turbine\SHARK-Turbine.venv\Lib\site-packages\transformers\models\convnext\feature_extraction_convnext.py:28: FutureWarning: The class ConvNextFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ConvNextImageProcessor instead.
  warnings.warn(


In [3]:
###
#standard inference example (no SHARK-Turbine):
###

#load an example
dataset = load_dataset("huggingface/cats-image")
image = dataset["test"]["image"][0]

#image.save("cats-image.jpg") #if you want to see the cute picture :)

#image2 = PIL.JpegImagePlugin.JpegImageFile("yourexamplepicture.jpg")

In [4]:
#extract features from image(s) to feed to model
inputs = extractor(image, return_tensors="pt")
#inputs2 = extractor(image2, return_tensors="pt")
pixel_tensor = inputs.pixel_values

In [5]:
#standard torch application of model to extracted feature input.

with torch.no_grad():
    logits = model.forward(pixel_tensor).logits
#    logits2 = model(**inputs2).logits

predicted_label = torch.argmax(logits,-1).item()
#predicted_label2 = logits2.argmax(-1).item()

print(model.config.id2label[predicted_label])
#print(model.config.id2label[predicted_label2])


tabby, tabby cat


In [6]:
###
#Using SHARK-Turbine to write a torch-mlir file for performing inference with the model.
###

class wtfamidoing2(CompiledModule):
    #this one is trying to pass the pixel_value tensor as an Abstract Tensor?
    params = export_parameters(model, initialize =False) #         is this necessary? 

    def forward(self, x = AbstractTensor(1, 3, 224, 224, dtype=torch.float32) ):
        with torch.no_grad():
            logits0 = jittable(model.forward)(x)['logits']
        label = self.labelout(logits0)
        #str_output = model.config.id2label[predicted_label0] 
        return label

    @jittable
    def labelout(logitsin):
        label =  torch.argmax(logitsin,-1)
        return label

In [7]:
#build an instance of our CompiledModule and get the corresponding mlir code
inst2 = wtfamidoing2(context=Context(), import_to="IMPORT")

In [8]:
#module_str2 = str(CompiledModule.get_mlir_module(inst2))
#print(module_str2)

In [9]:
session = Session()
ExportedModule = exporter.ExportOutput(session,inst2)

In [10]:
compiled_binary = ExportedModule.compile(save_to=None)

RuntimeError: Compilation failed: See diagnostics